# 步骤 1: 准备工作与环境设置

**目标:** 导入所有需要的库，并设置好文件路径和全局变量。

In [1]:
# --- 步骤 1: 准备工作与环境设置 ---

# --- 核心库 ---
import html
import json
import os
import re
import time
import unicodedata
from itertools import islice # 用于在测试模式下截取数据流
import shutil # 用于在服务器模式下高效复制文件

# --- 数据处理与NLP库 ---
import pandas as pd
import psutil # 用于智能检测CPU核心数
import spacy
from flashtext import KeywordProcessor # 用于高速初筛
from tqdm.auto import tqdm # 用于显示美观的进度条

# ==============================================================================
# --- 核心配置区 (通常您只需要修改这部分) ---
# ==============================================================================

# 1. 环境配置
# 目的: 自动适配不同运行环境，解决服务器上因网络存储(NFS)导致的I/O错误
#       或本地与服务器目录结构不一致的问题。
# 选项:
#   'local': 在您自己的电脑上运行。代码将使用相对路径 (如 ../data_raw)。
#   'dsw':   在阿里云DSW或其他远程服务器上运行。代码将使用绝对路径
#            (如 /mnt/data/data_raw)，并智能地将I/O密集型操作重定向
#            到服务器本地的 /tmp 目录，以获得最佳稳定性和速度。
RUNNING_ENV = 'local'

# 2. 快速测试模式开关
# 目的: 用于快速调试代码逻辑，避免处理全量数据耗费大量时间。
# 效果:
#   True:  只处理开头指定数量(TEST_SAMPLE_SIZE)的原始新闻和候选新闻，
#          可以快速验证整个流程是否通畅。
#   False: 处理所有数据，用于正式产出最终结果。
TEST_MODE = False
TEST_SAMPLE_SIZE = 50000        # 测试模式下，从源文件读取的最大行数
CANDIDATE_SAMPLE_SIZE = 2000    # 测试模式下，从候选集文件读取的最大行数

# ==============================================================================
# --- 路径智能管理 (根据上面的 RUNNING_ENV 自动配置，通常无需修改) ---
# ==============================================================================
print(f"检测到运行环境为: 【{RUNNING_ENV.upper()}】")
TEMP_DIR = '/tmp' # 定义服务器的本地临时目录

# 根据环境定义数据根路径
if RUNNING_ENV == 'local':
    # 本地模式: 使用相对于当前脚本(通常在 'scripts' 目录)的路径
    print("使用 'local' 模式的相对路径。")
    BASE_DATA_RAW_PATH = '../data_raw'
    BASE_DATA_PROCESSED_PATH = '../data_processed'
elif RUNNING_ENV == 'dsw':
    # DSW模式: 使用服务器上挂载存储的绝对路径
    print("使用 'dsw' 模式的绝对路径。")
    BASE_DATA_RAW_PATH = '/mnt/data/data_raw'
    BASE_DATA_PROCESSED_PATH = '/mnt/data/data_processed'
else:
    # 错误处理，防止配置错误
    raise ValueError(f"未知的 RUNNING_ENV: '{RUNNING_ENV}'. 请选择 'local' 或 'dsw'。")

# 使用上面定义的基础路径来构建“原始”或“最终”的文件路径
KEYWORD_JSON_PATH = os.path.join(BASE_DATA_RAW_PATH, 'china_keywords_collection.json')
SOURCE_NEWS_FILE_ORIGINAL = os.path.join(BASE_DATA_RAW_PATH, 'final_merged_all_news.csv')
CANDIDATES_FILE_ORIGINAL = os.path.join(BASE_DATA_PROCESSED_PATH, 'china_news_candidates.csv')
FINAL_RESULT_FILE = os.path.join(BASE_DATA_PROCESSED_PATH, 'final_china_news.csv')
REJECTED_FILE = os.path.join(BASE_DATA_PROCESSED_PATH, 'china_news_rejected_articles.csv')

# 初始化将要在后续流程中实际使用的路径变量，默认指向原始路径
SOURCE_NEWS_FILE = SOURCE_NEWS_FILE_ORIGINAL
CANDIDATES_FILE = CANDIDATES_FILE_ORIGINAL

# 如果是服务器环境，则执行I/O优化：重定向高负载的读写路径到 /tmp
if RUNNING_ENV == 'dsw':
    print("DSW 环境模式已激活，为避免I/O错误，将使用本地临时目录 /tmp ...")
    TEMP_SOURCE_NEWS_FILE = os.path.join(TEMP_DIR, 'final_merged_all_news.csv')
    TEMP_CANDIDATES_FILE = os.path.join(TEMP_DIR, 'china_news_candidates.csv')

    try:
        if not os.path.exists(SOURCE_NEWS_FILE_ORIGINAL):
             raise FileNotFoundError(f"在DSW的源路径 {SOURCE_NEWS_FILE_ORIGINAL} 未找到文件！")

        source_size = os.path.getsize(SOURCE_NEWS_FILE_ORIGINAL)
        temp_exists = os.path.exists(TEMP_SOURCE_NEWS_FILE)

        # 智能复制：仅在临时文件不存在或大小与源文件不一致时才执行复制操作
        if not temp_exists or os.path.getsize(TEMP_SOURCE_NEWS_FILE) != source_size:
            print(f"正在从 {SOURCE_NEWS_FILE_ORIGINAL} 复制到 {TEMP_SOURCE_NEWS_FILE} ... (这可能需要几分钟)")
            shutil.copy(SOURCE_NEWS_FILE_ORIGINAL, TEMP_SOURCE_NEWS_FILE)
            print("复制完成。")
        else:
            print(f"临时源文件 {TEMP_SOURCE_NEWS_FILE} 已存在且大小一致，跳过复制。")

        # [关键] 重定向路径变量，让后续代码块透明地使用/tmp下的文件
        SOURCE_NEWS_FILE = TEMP_SOURCE_NEWS_FILE
        CANDIDATES_FILE = TEMP_CANDIDATES_FILE

    except Exception as e:
        print(f"❌ 处理DSW临时文件时出错: {e}")
        # 如果出错，停止执行，因为在DSW上使用原始网络路径风险很高
        raise e

print("\n--- 最终文件路径配置 ---")
print(f"关键词文件 (读): {KEYWORD_JSON_PATH}")
print(f"源新闻文件 (读): {SOURCE_NEWS_FILE}")
print(f"候选集文件 (写/读): {CANDIDATES_FILE}")
print(f"最终结果文件 (写): {FINAL_RESULT_FILE}")
print(f"拒绝文件 (写): {REJECTED_FILE}")

# ==============================================================================
# --- 全局处理与积分系统配置 ---
# ==============================================================================

# --- 全局处理参数 ---
NEWS_COLUMN = 'CONTENT' # 定义新闻内容所在的列名
CHUNKSIZE = 20000       # 定义初筛时每个数据块的大小(行数)
BATCH_SIZE = 500        # 定义spaCy并行处理时每个批次的大小(文档数)

# --- 智能配置并行参数 ---
# 使用所有可用的物理CPU核心，并保留一个核心给操作系统，以避免系统卡顿
# logical=False 在物理机或高性能虚拟机上更稳健，避免超线程干扰
cpu_cores = psutil.cpu_count(logical=False)
N_PROCESSES = min(cpu_cores - 1 if cpu_cores > 1 else 1, 8) # 最多使用8个进程，防止资源过度消耗
if N_PROCESSES < 1: N_PROCESSES = 1

# --- 精筛积分系统配置 (核心调优区域) ---
# 调优思路: 通过高额的"前导奖励"和"关键词频率分"确保高价值文章能获得远超
#           阈值的分数，使其足以抵御少量负面信号（如假设句）的扣分。

# 1. 最终决策阈值
# 描述: 一篇文章的最终总分必须高于此阈值才会被接受。这是调节筛选器
#       宽松度的最主要杠杆。调低会增加通过率，调高则更严格。
ACCEPTANCE_THRESHOLD = 5.0

# 2. 正向加分项 (奖励相关信号)
# 描述: 奖励那些在文章开头(前10句)就点明主题的文章。
#       根据开头出现的最高级别关键词给予不同奖励。
LEAD_BONUS_TIER_5 = 20.0 # T5词在开头，几乎是决定性的
LEAD_BONUS_TIER_4 = 15.0
LEAD_BONUS_TIER_3 = 10.0
LEAD_BONUS_TIER_2 = 5.0  # T2词在开头，也是一个强信号

# 描述: 奖励那些通篇都在讨论相关话题的文章，衡量其“相关深度”。
#       高等级的词给予不成比例的高分，以体现其重要性。
TIER_5_SCORE = 5.0       # 例如:
TIER_4_SCORE = 4.0       # 例如:
TIER_3_SCORE = 3.0       # 例如:
TIER_2_SCORE = 2.0       # 例如:
TIER_1_SCORE = 1.0       # 例如:

# 3. 负向扣分项 (惩罚风险信号)
# 描述: 惩罚那些可能引入不相关信息的上下文。
NEGATION_PENALTY = -3.0     # 例如: "not about China"
HYPOTHETICAL_PENALTY = -2.0 # 例如: "if China were to..."

# ==============================================================================
# --- 启动信息 ---
# ==============================================================================
print("\n✅ 块 1: 库导入和配置完成。")
print("-" * 30)
if TEST_MODE:
    print(f"🚀🚀🚀 运行在【快速测试模式】下！🚀🚀🚀")
else:
    print("🚢🚢🚢 运行在【完整数据模式】下。🚢🚢🚢")
print(f"   - 精筛将采用升级版5级积分制，接受阈值为: {ACCEPTANCE_THRESHOLD} 分")
print("-" * 30)
print(f"   - 初筛将使用单进程顺序处理。")
print(f"   - 精筛阶段将使用 {N_PROCESSES} 个进程进行并行处理。")

检测到运行环境为: 【LOCAL】
使用 'local' 模式的相对路径。

--- 最终文件路径配置 ---
关键词文件 (读): ../data_raw\china_keywords_collection.json
源新闻文件 (读): ../data_raw\final_merged_all_news.csv
候选集文件 (写/读): ../data_processed\china_news_candidates.csv
最终结果文件 (写): ../data_processed\final_china_news.csv
拒绝文件 (写): ../data_processed\china_news_rejected_articles.csv

✅ 块 1: 库导入和配置完成。
------------------------------
🚢🚢🚢 运行在【完整数据模式】下。🚢🚢🚢
   - 精筛将采用升级版5级积分制，接受阈值为: 5.0 分
------------------------------
   - 初筛将使用单进程顺序处理。
   - 精筛阶段将使用 5 个进程进行并行处理。


# 步骤 2: 初筛准备 - 构建智能正则表达式

**目标:** 读取 中国相关关键词 JSON 文件，并执行我们讨论过的所有逻辑来构建一个强大、高效的正则表达式。

In [2]:
# --- 构建初筛用的 Flashtext 关键词处理器 ---
def build_keyword_processor(json_path):
    print(f"正在从 {json_path} 加载关键词...")
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            keywords_data = json.load(f)
    except FileNotFoundError:
        print(f"❌ 错误: 关键词文件未找到 {json_path}")
        return None, None
    print(f"共加载 {len(keywords_data)} 个关键词对象。")
    all_aliases = set()
    for item in keywords_data:
        all_aliases.add(item['keyword'])
        for alias in item.get('aliases', []):
            all_aliases.add(alias)
    print(f"提取出 {len(all_aliases)} 个不重复的关键词/别名。")
    keyword_processor = KeywordProcessor(case_sensitive=False)
    for kw in all_aliases:
        keyword_processor.add_keyword(kw)
    print("✅ 高效关键词处理器 (Flashtext) 构建完成。")
    return keyword_processor, keywords_data

keyword_processor, keywords_data = build_keyword_processor(KEYWORD_JSON_PATH)
print("\n✅ 块 2: 初筛准备工作完成。")

正在从 ../data_raw\china_keywords_collection.json 加载关键词...
共加载 274 个关键词对象。
提取出 394 个不重复的关键词/别名。
✅ 高效关键词处理器 (Flashtext) 构建完成。

✅ 块 2: 初筛准备工作完成。


# 步骤 3: 执行阶段一 - 调用外部脚本进行快速初筛

**目标:** 对大文件进行分块扫描，应用正则表达式，并保存候选集。这将是整个流程中最耗时的部分。

In [3]:
# --- 步骤 3: 执行第一阶段 - 大规模流式初筛 (已优化进度条) ---
def lightweight_clean(text):
    if not isinstance(text, str): return ""
    text = re.sub('<[^>]*>', '', text)
    text = html.unescape(text)
    text = unicodedata.normalize('NFKC', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def process_chunk(chunk_df):
    if NEWS_COLUMN not in chunk_df.columns: return pd.DataFrame()
    cleaned_series = chunk_df[NEWS_COLUMN].astype(str).apply(lightweight_clean)
    mask = cleaned_series.apply(lambda x: len(keyword_processor.extract_keywords(x)) > 0)
    return chunk_df[mask]

print("--- 阶段一: 开始使用单进程进行流式初筛 ---")
start_time = time.time()
try:
    total_chunks = None
    if not TEST_MODE:
        print("正在计算文件总块数 (优化方式)...")
        try:
            first_col_name = pd.read_csv(SOURCE_NEWS_FILE, nrows=0).columns[0]
            row_iterator = pd.read_csv(SOURCE_NEWS_FILE, chunksize=100000, usecols=[first_col_name], on_bad_lines='skip', low_memory=False)
            num_lines = sum(len(chunk) for chunk in tqdm(row_iterator, desc="预扫描文件行数"))
            total_chunks = (num_lines // CHUNKSIZE) + 1 if CHUNKSIZE > 0 else 1
            print(f"文件包含 {num_lines} 有效行, 约 {total_chunks} 个数据块。")
        except Exception as e:
            print(f"快速计算行数失败: {e}. 将不显示总进度。")

    chunk_iterator = pd.read_csv(SOURCE_NEWS_FILE, chunksize=CHUNKSIZE, on_bad_lines='skip', low_memory=False)

    if TEST_MODE:
        num_test_chunks = (TEST_SAMPLE_SIZE // CHUNKSIZE) + 1 if CHUNKSIZE > 0 else 1
        chunk_iterator = islice(chunk_iterator, num_test_chunks)
        total_chunks = num_test_chunks
        print(f"🚀 测试模式: 将处理前 {total_chunks} 个数据块 (约 {TEST_SAMPLE_SIZE} 行)。")

    print("将使用单进程顺序处理...")
    is_first_chunk = True
    total_candidates = 0

    for chunk_df in tqdm(chunk_iterator, total=total_chunks, desc="顺序初筛中"):
        candidates_df = process_chunk(chunk_df)
        if not candidates_df.empty:
            total_candidates += len(candidates_df)
            if is_first_chunk:
                candidates_df.to_csv(CANDIDATES_FILE, index=False, mode='w', encoding='utf-8')
                is_first_chunk = False
            else:
                candidates_df.to_csv(CANDIDATES_FILE, index=False, mode='a', header=False, encoding='utf-8')

    end_time_stage1 = time.time()
    print(f"\n--- 初筛流程执行完毕 ---\n总共找到 {total_candidates} 篇候选文章，已保存到 {CANDIDATES_FILE}")
    print(f"阶段一 (顺序初筛) 耗时: {(end_time_stage1 - start_time) / 60:.2f} 分钟。")
except FileNotFoundError:
    print(f"❌ 错误: 原始新闻文件未找到 {SOURCE_NEWS_FILE}")
except Exception as e:
    print(f"❌ 处理过程中发生错误: {e}")

print("\n✅ 块 3: 初筛流程执行完毕。")

--- 阶段一: 开始使用单进程进行流式初筛 ---
正在计算文件总块数 (优化方式)...


预扫描文件行数: 0it [00:00, ?it/s]

文件包含 1560878 有效行, 约 79 个数据块。
将使用单进程顺序处理...


顺序初筛中:   0%|          | 0/79 [00:00<?, ?it/s]

KeyboardInterrupt: 

# 步骤 4: 精筛准备 - 加载模型与定义规则

**目标:** 负责加载 spaCy 模型和数据，并定义所有用于精筛的“否决规则”函数。

In [ ]:
# --- 步骤 4: 准备第二阶段 - 构建多维度相关性评估引擎 (已升级为5级评分) ---
print("--- 阶段二准备: 加载 spaCy 及构建评估规则 ---")
try:
    nlp = spacy.load("en_core_web_lg", disable=["ner", "lemmatizer", "tagger", "attribute_ruler"])
    print(f"✅ spaCy 模型 '{nlp.meta['name']}' 的核心组件加载成功。")
except OSError:
    print("错误: spaCy模型 'en_core_web_lg' 未安装。请运行: python -m spacy download en_core_web_lg")
    nlp = None

if nlp:
    print("正在构建关键词信息查找表...")
    keyword_lookup = {}
    for item in keywords_data:
        tier = item.get('relevance_tier', 1)
        keyword_lookup[item['keyword'].lower()] = {'tier': tier}
        for alias in item.get('aliases', []):
            keyword_lookup[alias.lower()] = {'tier': tier}
    print(f"✅ 查找表构建完成，包含 {len(keyword_lookup)} 个词条。")

    print("正在准备 PhraseMatcher...")
    from spacy.matcher import PhraseMatcher
    matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
    patterns = [nlp.make_doc(text) for text in keyword_lookup.keys()]
    matcher.add("ChinaKeywords", patterns)
    print(f"✅ PhraseMatcher 准备完成，已添加 {len(patterns)} 个模式。")

    def score_keyword_frequency(matches, doc, lookup):
        score = 0
        for match_id, start, end in matches:
            kw = doc[start:end].text.lower()
            tier = lookup.get(kw, {}).get('tier', 1)
            if tier == 5: score += TIER_5_SCORE
            elif tier == 4: score += TIER_4_SCORE
            elif tier == 3: score += TIER_3_SCORE
            elif tier == 2: score += TIER_2_SCORE
            else: score += TIER_1_SCORE
        return score

    def score_lead_paragraphs_presence(doc, matcher, lookup):
        sents = list(doc.sents)
        if not sents: return 0, ""
        lead_sents_count = min(len(sents), 10)
        lead_end_token_index = sents[lead_sents_count - 1].end
        matches = matcher(doc)
        highest_tier_in_lead = 0
        found_kw = ""
        for match_id, start, end in matches:
            if start < lead_end_token_index:
                kw = doc[start:end].text.lower()
                tier = lookup.get(kw, {}).get('tier', 1)
                if tier > highest_tier_in_lead:
                    highest_tier_in_lead = tier
                    found_kw = kw
        if highest_tier_in_lead == 5: return LEAD_BONUS_TIER_5, f"前导加分-T5(+'{found_kw}')"
        if highest_tier_in_lead == 4: return LEAD_BONUS_TIER_4, f"前导加分-T4(+'{found_kw}')"
        if highest_tier_in_lead == 3: return LEAD_BONUS_TIER_3, f"前导加分-T3(+'{found_kw}')"
        if highest_tier_in_lead == 2: return LEAD_BONUS_TIER_2, f"前导加分-T2(+'{found_kw}')"
        return 0, ""

    def penalize_hypothetical(doc, keywords_in_doc):
        penalty = 0
        reasons = []
        for sent in doc.sents:
            clean_sent_start = sent.text.strip().lower()
            if clean_sent_start.startswith(('if ', 'unless ', 'what if')):
                if any(token.text.lower() in keywords_in_doc for token in sent):
                    penalty += HYPOTHETICAL_PENALTY
                    reasons.append(f"假设句扣分: '{sent.text[:50].strip()}...'")
        return penalty, reasons

    def penalize_negation(doc, keywords_in_doc):
        penalty = 0
        reasons = []
        for token in doc:
            if token.dep_ == "neg" and token.head.text.lower() in keywords_in_doc:
                penalty += NEGATION_PENALTY
                reasons.append(f"否定扣分: '{token.text} {token.head.text}'")
        return penalty, reasons

    print("✅ 块 4: 精筛规则和评估引擎准备完成 (已升级为5级评分)。")

# 步骤 5: 执行阶段二 - 精筛流程

**目标:** 加载候选集，应用所有否决规则，然后保存最终结果和被拒绝的文章。

In [ ]:
# --- 步骤 5: 执行第二阶段 - 上下文精筛与产出 (最终版，已适配DSW多核) ---
print("--- 阶段二: 开始精筛候选集 ---")
start_time_s2 = time.time()
try:
    read_csv_kwargs = {'low_memory': False}
    if TEST_MODE:
        read_csv_kwargs['nrows'] = CANDIDATE_SAMPLE_SIZE
        print(f"🚀 测试模式: 最多加载前 {CANDIDATE_SAMPLE_SIZE} 篇候选文章进行精筛。")
    df_candidates = pd.read_csv(CANDIDATES_FILE, **read_csv_kwargs)
    print(f"✅ 成功加载 {len(df_candidates)} 篇候选文章。")
except FileNotFoundError:
    print(f"❌ 错误: 候选文件未找到 {CANDIDATES_FILE}。请先运行块 3。")
    df_candidates = pd.DataFrame()

if not df_candidates.empty and nlp:
    # DSW优化1: 将Pandas Series转为Python原生list，提高跨进程序列化的稳定性和速度
    texts = df_candidates[NEWS_COLUMN].astype(str).tolist()
    results = []
    print(f"开始使用 {N_PROCESSES} 个进程并行处理文本...")

    # DSW优化2: 使用 with nlp.select_pipes() 上下文管理器。
    # 这是在 DSW 等环境中实现稳定多进程的关键。
    # 它能确保子进程只加载和使用当前任务必需的组件，避免进程卡死。
    # 我们需要 'senter' (用于 doc.sents) 和 'parser' (用于 token.dep_)
    with nlp.select_pipes(enable=["senter", "parser"]):
        docs_content = nlp.pipe(texts, batch_size=BATCH_SIZE, n_process=N_PROCESSES)

        # 将迭代处理放在 'with' 块内部，以确保在多进程上下文中执行
        for doc in tqdm(docs_content, total=len(df_candidates), desc="精筛文章"):
            score = 0
            score_details = []
            matches = matcher(doc)
            if not matches:
                results.append({'score': 0, 'reason': '精筛阶段未匹配到任何关键词'})
                continue

            # 业务逻辑保持不变，依然使用5级评分系统
            found_keywords_text = {doc[start:end].text.lower() for _, start, end in matches}

            lead_score, lead_reason = score_lead_paragraphs_presence(doc, matcher, keyword_lookup)
            if lead_score > 0: score_details.append(lead_reason)

            freq_score = score_keyword_frequency(matches, doc, keyword_lookup)
            if freq_score > 0: score_details.append(f"关键词频率分: +{freq_score:.2f}")

            score = lead_score + freq_score

            hypo_penalty, hypo_reasons = penalize_hypothetical(doc, found_keywords_text)
            nega_penalty, nega_reasons = penalize_negation(doc, found_keywords_text)

            score += hypo_penalty
            score += nega_penalty

            score_details.extend(hypo_reasons)
            score_details.extend(nega_reasons)

            results.append({'score': score, 'reason': ' | '.join(score_details)})

    # 以下结果合并与保存部分，在 'with' 块之外执行
    print("\n正在合并精筛结果...")
    df_results = pd.DataFrame(results, index=df_candidates.index)
    df_results['keep'] = df_results['score'] >= ACCEPTANCE_THRESHOLD
    df_final_with_reasons = pd.concat([df_candidates, df_results], axis=1)
    df_accepted = df_final_with_reasons[df_final_with_reasons['keep'] == True].drop(columns=['keep', 'score', 'reason'])
    df_rejected = df_final_with_reasons[df_final_with_reasons['keep'] == False].drop(columns=['keep'])

    print("\n--- 精筛完成 ---")
    df_accepted.to_csv(FINAL_RESULT_FILE, index=False, encoding='utf-8')
    print(f"✅ {len(df_accepted)} 篇最终文章已保存到: {FINAL_RESULT_FILE}")
    df_rejected.to_csv(REJECTED_FILE, index=False, encoding='utf-8')
    print(f"ℹ️ {len(df_rejected)} 篇被拒绝的文章已保存到: {REJECTED_FILE} (供分析)")

    end_time_s2 = time.time()
    total_minutes = (end_time_s2 - start_time_s2) / 60
    print(f"阶段二 (精筛) 耗时: {total_minutes:.2f} 分钟。")
else:
    print("候选集为空或spaCy模型未加载，跳过精筛。")

print("\n✅ 块 5: 精筛流程执行完毕。")